In [ ]:
!pip install scikit-plot

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # Name for the column  or Features Map
columns_to_named = ["frameNumber", "frameTime", "frameLen", "ethSrc", "ethDst",
       "ipSrc", "ipDst", "ipProto", "ipLen", "tcpLen", "tcpSrcport",
       "tcpDstport", "Value", "normality"]
    
    # Read the Dataset and Rename the Column
df = pd.read_csv("/content/drive/My Drive/Code/dataset/Preprocessed_data.csv",header=0,names=columns_to_named)

col_norm = ['frameNumber', 'frameTime', 'frameLen', 'ethSrc', 'ethDst',
       'ipSrc', 'ipDst', 'ipProto', 'ipLen', 'tcpLen', 'tcpSrcport',
       'tcpDstport', 'Value']
    
X_Data = df[col_norm]
y = df["normality"]
#normalize
sc = StandardScaler()
X = sc.fit_transform(X_Data)

y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5])
n_classes = y.shape[1]
print(n_classes)

df.shape
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=1)


6


In [ ]:
target_names = ['Normal', 'WrongSetup', 'DDoS', 'Probing', 'Scan', 'MITM']

In [ ]:
#decision tree with 5 fold cv
from sklearn import tree
pipe_tree = Pipeline([('clf', OneVsRestClassifier(tree.DecisionTreeClassifier(max_depth=3)))])
train_sizes_tree, train_scores_tree, test_scores_tree = learning_curve(estimator=pipe_tree,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_tree = np.mean(train_scores_tree, axis=1)
train_std_tree = np.std(train_scores_tree, axis=1)
test_mean_tree = np.mean(test_scores_tree, axis=1)
test_std_tree = np.std(test_scores_tree, axis=1)

In [ ]:
for i in train_mean_tree:
    print(i)

0.9980101586636645
0.9962978478295019
0.9980520500602188
0.9964994501754202
0.99742577368173


In [ ]:
for i in test_mean_tree:
    print(i)

0.8878253250727196
0.7879918433740235
0.8279918388113595
0.7264787747272349
0.8597874389799504


In [ ]:
#dt training
pipe_tree = pipe_tree.fit(X_train, y_train)
y_pred_test = pipe_tree.predict(X_test)
y_pred_train=pipe_tree.predict(X_train)
# print dt_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))

# print dt_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
#dt classification report
print(classification_report(y_train, y_pred_train, target_names=target_names))



Test_Accuracy: 0.9981882160735606
Test_Precision: 1.0
Test_Recall: 0.9981882160735606
Test_f1-score: 0.9990914543139499
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     15770
  WrongSetup       1.00      1.00      1.00     16498
        DDoS       1.00      1.00      1.00     15841
     Probing       1.00      0.99      1.00     15947
        Scan       1.00      1.00      1.00     15851
        MITM       1.00      1.00      1.00     15579

   micro avg       1.00      1.00      1.00     95486
   macro avg       1.00      1.00      1.00     95486
weighted avg       1.00      1.00      1.00     95486
 samples avg       1.00      1.00      1.00     95486



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_Accuracy: 0.9979447033565482
Train_Precision: 1.0
Train_Recall: 0.9979447033565482
Train_f1-score: 0.9989688901481217
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     63265
  WrongSetup       1.00      1.00      1.00     65787
        DDoS       1.00      1.00      1.00     63179
     Probing       1.00      0.99      1.00     63055
        Scan       1.00      1.00      1.00     63201
        MITM       1.00      1.00      1.00     63453

   micro avg       1.00      1.00      1.00    381940
   macro avg       1.00      1.00      1.00    381940
weighted avg       1.00      1.00      1.00    381940
 samples avg       1.00      1.00      1.00    381940



In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
pipe_rnd = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(max_depth=3, n_estimators=100)))])
train_sizes_rnd, train_scores_rnd, test_scores_rnd = learning_curve(estimator=pipe_rnd,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_rnd = np.mean(train_scores_rnd, axis=1)
train_std_rnd = np.std(train_scores_rnd, axis=1)
test_mean_rnd = np.mean(test_scores_rnd, axis=1)
test_std_rnd = np.std(test_scores_rnd, axis=1) 

In [ ]:
for i in train_mean_tree:
    print(i)

0.9980101586636645
0.9962978478295019
0.9980520500602188
0.9964994501754202
0.99742577368173


In [ ]:
for i in test_mean_tree:
    print(i)

0.8878253250727196
0.7879918433740235
0.8279918388113595
0.7264787747272349
0.8597874389799504


In [ ]:
#rf training
pipe_rnd = pipe_rnd.fit(X_train, y_train)
y_pred_test = pipe_rnd.predict(X_test)
y_pred_train=pipe_rnd.predict(X_train)
# print rf_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))


# print rf_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
print(classification_report(y_train, y_pred_train, target_names=target_names))




Test_Accuracy: 0.9980625432000503
Test_Precision: 0.999540439638634
Test_Recall: 0.9985128709967953
Test_f1-score: 0.9990246582421971
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     15770
  WrongSetup       1.00      1.00      1.00     16498
        DDoS       1.00      1.00      1.00     15841
     Probing       1.00      1.00      1.00     15947
        Scan       1.00      1.00      1.00     15851
        MITM       1.00      1.00      1.00     15579

   micro avg       1.00      1.00      1.00     95486
   macro avg       1.00      1.00      1.00     95486
weighted avg       1.00      1.00      1.00     95486
 samples avg       1.00      1.00      1.00     95486



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_Accuracy: 0.9978216473791695
Train_Precision: 0.9995247597393623
Train_Recall: 0.9982850709535529
Train_f1-score: 0.9989023352796718
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     63265
  WrongSetup       1.00      1.00      1.00     65787
        DDoS       1.00      1.00      1.00     63179
     Probing       1.00      0.99      1.00     63055
        Scan       1.00      1.00      1.00     63201
        MITM       1.00      1.00      1.00     63453

   micro avg       1.00      1.00      1.00    381940
   macro avg       1.00      1.00      1.00    381940
weighted avg       1.00      1.00      1.00    381940
 samples avg       1.00      1.00      1.00    381940



In [ ]:
# svm
from sklearn.svm import SVC
pipe_svc = Pipeline([('clf', OneVsRestClassifier(SVC(C=1000)))])
train_sizes_svc, train_scores_svc, test_scores_svc = learning_curve(estimator=pipe_svc,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_svc = np.mean(train_scores_svc, axis=1)
train_std_svc = np.std(train_scores_svc, axis=1)
test_mean_svc = np.mean(test_scores_svc, axis=1)
test_std_svc = np.std(test_scores_svc, axis=1)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
for i in train_mean_svc:

    print(i)

0.9979813583285333
0.9987196941928052
0.9981358328533277
0.9983308896685343
0.9986102529193067


In [ ]:
for i in test_mean_svc:
    print(i)

0.8594590621075193
0.9104634836128982
0.8479077108132428
0.8817434017691494
0.8273432344229221


In [ ]:
#svm training
pipe_svc = pipe_svc.fit(X_train, y_train)
y_pred_test = pipe_svc.predict(X_test)
y_pred_train=pipe_svc.predict(X_train)
# print svc_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))


# print svc_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
print(classification_report(y_train, y_pred_train, target_names=target_names))




Test_Accuracy: 0.998439561820581
Test_Precision: 0.9984629203138901
Test_Recall: 0.999329744674612
Test_f1-score: 0.9988924645107707
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     15770
  WrongSetup       0.99      1.00      1.00     16498
        DDoS       1.00      1.00      1.00     15841
     Probing       1.00      1.00      1.00     15947
        Scan       1.00      1.00      1.00     15851
        MITM       1.00      1.00      1.00     15579

   micro avg       1.00      1.00      1.00     95486
   macro avg       1.00      1.00      1.00     95486
weighted avg       1.00      1.00      1.00     95486
 samples avg       1.00      1.00      1.00     95486



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_Accuracy: 0.9982484159815678
Train_Precision: 0.9982707665089718
Train_Recall: 0.9992354820128816
Train_f1-score: 0.9987480127033858
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     63265
  WrongSetup       0.99      1.00      1.00     65787
        DDoS       1.00      1.00      1.00     63179
     Probing       1.00      1.00      1.00     63055
        Scan       1.00      1.00      1.00     63201
        MITM       1.00      1.00      1.00     63453

   micro avg       1.00      1.00      1.00    381940
   macro avg       1.00      1.00      1.00    381940
weighted avg       1.00      1.00      1.00    381940
 samples avg       1.00      1.00      1.00    381940

